In [1]:
import pandas as pd
import glob as gb
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer, TfidfTransformer
from nltk.stem import WordNetLemmatizer, PorterStemmer

In [2]:
TrnFlsPth=gb.glob("DTrain/*/*")
TstFlsPth=gb.glob("DTest/*/*")

In [3]:
Tr_lbl=[e.strip().split('\\')[1] for e in TrnFlsPth]
Ts_lbl=[e.strip().split('\\')[1] for e in TstFlsPth]

In [4]:
Y_Trn=pd.get_dummies(Tr_lbl)
Y_Tst=pd.get_dummies(Ts_lbl)

In [5]:
fpt=[open(f) for f in TrnFlsPth]
Trfob=[f.read().replace('\n', ' ') for f in fpt]
all(f.close() for f in fpt)

False

In [6]:
fpt=[open(f) for f in TstFlsPth]
Tsfob=[f.read().replace('\n', ' ') for f in fpt]
all(f.close() for f in fpt)
fpt=None

In [7]:
vc=CountVectorizer(analyzer='word', binary=False, lowercase=True, max_df=10, min_df=1,
                   stop_words='english')
vc_ngm=CountVectorizer(analyzer='word', binary=False, lowercase=True, max_df=10, min_df=1,
                   ngram_range=(1, 2), stop_words='english')
idft=TfidfTransformer()
idft_ngm=TfidfTransformer()

In [8]:
TrDvcX=vc.fit_transform(Trfob)
TrDvc_ngmX=vc_ngm.fit_transform(Trfob)
TrDtvcX=idft.fit_transform(TrDvcX.toarray())
TrDtvc_ngmX=idft_ngm.fit_transform(TrDvc_ngmX.toarray())

In [9]:
TsDvcX=vc.transform(Tsfob)
TsDvc_ngmX=vc_ngm.transform(Tsfob)
TsDtvcX=idft.transform(TsDvcX.toarray())
TsDtvc_ngmX=idft_ngm.transform(TsDvc_ngmX.toarray())

In [23]:
TrDvc=pd.DataFrame(TrDvcX.toarray(), columns=vc.get_feature_names())
TrDvc_ng=pd.DataFrame(TrDvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())
TrDtvc=pd.DataFrame(TrDtvcX.toarray(), columns=vc.get_feature_names())
TrDtvc_ngm=pd.DataFrame(TrDtvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())

In [24]:
TsDvc=pd.DataFrame(TsDvcX.toarray(), columns=vc.get_feature_names())
TsDvc_ng=pd.DataFrame(TsDvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())
TsDtvc=pd.DataFrame(TsDtvcX.toarray(), columns=vc.get_feature_names())
TsDtvc_ngm=pd.DataFrame(TsDtvc_ngmX.toarray(), columns=vc_ngm.get_feature_names())

In [25]:
pd.set_option('display.max_row',10)
#pd.set_option('display.max_column',100)
#pd.reset_option('display.max_row')
#pd.reset_option('display.max_column')

In [26]:
TrDvcS=pd.concat([TrDvc, Y_Trn], axis=1)
TrDvcS

,00,000,002,003,0037,004,005,009,01,011,...,alum,barley,coffee,dmk,fuel,livestock,palm-oil,retail,soybean,veg-oil
0,0,3,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
495,0,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
497,0,11,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [19]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, cross_val_predict
#from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB 

In [20]:
clf=MultinomialNB()

In [32]:
clf.fit(TrDtvc, Tr_lbl)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [33]:
y_pred=clf.predict(TsDtvc)

In [34]:
confusion_matrix( Ts_lbl, y_pred)

array([[ 2,  0, 14,  0,  0,  1,  0,  0,  4,  2],
       [ 0,  5,  0,  0,  0,  0,  0,  0,  7,  2],
       [ 0,  0, 28,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  3,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  8,  0,  0,  0,  0,  0,  0,  2],
       [ 0,  0,  1,  0,  0, 18,  0,  0,  1,  4],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  9],
       [ 0,  0,  2,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  0,  1,  0,  0, 21,  6],
       [ 0,  0,  3,  0,  0,  0,  0,  0,  7, 27]], dtype=int64)

In [ ]:
ps=PorterStemmer()
Trfob_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob]
Tsfob_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Tsfob]

In [ ]:
lm=WordNetLemmatizer()
Trfob_lmt=[' '.join([lm.stem(word) for word in text.split(' ')]) for text in Trfob]
Tsfob_lmt=[' '.join([lm.stem(word) for word in text.split(' ')]) for text in Trfob]
Trfob_lmt_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob_lmt]
Tsfob_lmt_stm=[' '.join([ps.stem(word) for word in text.split(' ')]) for text in Trfob_lmt]